In [63]:
import numpy as np
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import random

# CLassificadores:

## Regressão Logística

### Funções auxiliares

In [64]:
def sigmoide(Z):
    return 1 / (1 + np.exp((-1 * Z)))

In [65]:
def GradienteDescendente(X, Y, epocas, alpha):
    Erros = []
    
    #Cria um array para 
    W = np.zeros((X.shape[1], 1))
    
    for t in range(0, epocas):
        #Calculo do erro na iteração t
        Y_ = (X @ W)
        Erro  = Y - sigmoide(Y_)
        Erros.append(np.mean(Erro))
        
        #Ajuste dos pesos na iteração t
        X_ = Erro * X
        Media = np.mean((Erro), axis = 0, keepdims=True)
        W = W + alpha * Media.T
        
    return (W, Y_, Erros)

In [66]:
def kfold(qtd_linhas, seed = 42, k = 10):
    #Cria uma lista com os indices do dataset
    indices_dataset = list(range(qtd_linhas))
    
    #Define o tamanho do dataset
    tamanho_subset = round(len(indices_dataset) / k)
    
    #Cria uma lista de subconjuto de indices
    random.Random(42).shuffle(indices_dataset)
    lista_subsets = [indices_dataset[x : x + tamanho_subset] for x in range(0, len(indices_dataset), tamanho_subset)]

    kfolds = []
    for i in range(k):
        teste = lista_subsets[i]
        treino = []
        for subset in lista_subsets:
            if subset != teste:
                treino.append(subset)
        kfolds.append((teste, treino))
        
    return kfolds

In [67]:
def montar_instancias(lista_indices, data_set, treino = True):
    instancias = []
    
    if treino:
        for lista in lista_indices:
            for indice in lista:
                instancias.append(data_set[indice])
    else:
        for indice in lista_indices:
            instancias.append(data_set[indice])
            
    return np.array(instancias)

### Função Principal

In [68]:
data_set = np.genfromtxt('breastcancer.csv', delimiter = ',')
k = 10

#Por questão de legibilidade
qtd_linhas, qtd_colunas = data_set.shape
data_set = np.c_[np.ones((1, qtd_linhas)).T, data_set]

#Calcula os conjuntos de teste e treino
kfolds = kfold(qtd_linhas) 

testes = []
treinos = []
for teste, treino in kfolds:
    testes.append(montar_instancias(teste, data_set,  treino=False))
    treinos.append(montar_instancias(treino, data_set, treino=True))
    
resultado = []
for i in range(0, k):
    
    menor_erro = 10
    for treino in treinos:
        X_treino = treino[:, :-1]
        Y_treino = treino[:, [-1]]
        W, Y_, Erros = GradienteDescendente(X_treino, Y_treino, 2000, 0.001)
        
        if Erros[-1] < menor_erro:
            menor_erro = Erros[-1]
            melhor_w = W
            
    # resultado_treino.append((melhor_w, Erros[-1]))
    X_teste = testes[i][:, :-1]
    Y_teste = testes[i][:, [-1]]
    
    Y_ = X_teste @ melhor_w
    erro_teste = np.mean(( Y_teste - sigmoide(Y_)))
    
    resultado.append((melhor_w, Erros[-1], erro_teste))

### Analise dos resultados:

## Analise do Discriminante Gaussiano

In [106]:
#Por questões de legibilidade
qtd_linhas, qtd_colunas = data_set.shape

def calcula_numero_classe(data_set):
    #Conta a quantidade de classes, cada posição no vetor representa uma classe
    N_classes = [0, 0]
    for inf in data_set:
        if inf[-1] == 0.0:
            N_classes[0] += 1
        else:
            N_classes[1] += 1
    return N_classes

def calcula_prob_classe(data_set, N_classe):
    N = data_set.shape[0]
    # Prob_classe é um vetor onde cada posíção representa uma classe e o valor associado a probabilidade
    pob_classe = [0, 0]
    for c in range(0, Nk):
        pob_classe[c] = N_classes[c] / N
        
    return pob_classe

def calcula_medias_classe(data_set):
    #Calculando Médias das classes:
    medias = np.zeros(2)
    for inf in data_set:
        if inf[-1] == 0.0:
            medias[0] += inf[:-1].sum()
        else:
            medias[1] += inf[:-1].sum()

    medias[0] = medias[0] / N_classes[0]
    medias[1] = medias[1] / N_classes[1]
    
    return medias

def calcula_sigmas_classe(data_set, medias, N_classes):
    #Calculando os desvios padrão de cada classe
    sigmas = [0, 0]
    for inf in data_set:
        if inf[-1] == 0:
            sigmas[0] += (inf[:-1] - medias[0]) @ (inf[:-1] - medias[0]).T
        else:
            sigmas[1] += (inf[:-1] - medias[1]) @ (inf[:-1] - medias[1]).T
    
    for j in range(2):
        aux = np.zeros(4)
        
        for i in range(4):
            if i == 0 or i == 3:
                aux[i] = (sigmas[j] / (N_classes[j] - 1)) ** 2
            else:
                aux[i] = (sigmas[j] / (N_classes[j] - 1))
            
        sigmas[j] = aux
        
    return np.array(sigmas)

In [107]:
def discriminante_gaussiano(prob_classe, N_classe, medias, sigmas, X):
    Y_ = np.zeros(2)
    
    for i in range(0, 2):
        Y_[i] = np.log(prob_classe[i]) @ ((-1/2) * np.log(sigmas[i])) @ ((-1/2) * (X - medias[i]).T) @ np.linalg.inv(sigmas[i]) @ (X - medias[i])
        
    return Y_.argmax()

### Função Principal


In [108]:
data_set = np.genfromtxt('breastcancer.csv', delimiter = ',')

#Calcula os conjuntos de teste e treino
kfolds = kfold(qtd_linhas) 

testes = []
treinos = []
for teste, treino in kfolds:
    testes.append(montar_instancias(teste, data_set,  treino=False))
    treinos.append(montar_instancias(treino, data_set, treino=True))
    
resultado = []
for i in range(0, k):
    for treino in treinos:
        N_classe = calcula_numero_classe(treino)
        prob_classe = calcula_prob_classe(treino, N_classe)
        medias = calcula_medias_classe(treino)
        sigmas = calcula_sigmas_classe(treino, medias, N_classes)
        
    # 
    X_teste = testes[i][:, :-1]
    Y_teste = testes[i][:, [-1]]
    
    Y_ = discriminante_gaussiano(prob_classe, N_classe, medias, sigmas, X_teste)
    
    resultado.append(Y_)

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [71]:
teste = calcula_sigmas_classe(data_set)

In [72]:
teste

array([2.36756215e+08, 4.82979012e+07])

In [76]:
teste = (zip([0, 1], [0, 1, 2, 3]))

In [77]:
teste

In [81]:
for i, j in enumerate([0, 1, 2, 3]):
    print(i, j)

0 0
1 1
2 2
3 3


In [90]:
np.log().amax()


AttributeError: 'numpy.ndarray' object has no attribute 'amax'

In [97]:
np.array([1,2,3]).argmax(axis=0)

2